In [1]:
import torch
import torch.nn as nn

In [2]:
# q: [b,s,n_heads,d_head] and k: [b,s,n_kv_heads,d_head]

In [3]:
q = torch.randn(1, 5, 12, 768)
k = torch.randn(1, 5, 4, 768)

In [4]:
q.shape, k.shape

(torch.Size([1, 5, 12, 768]), torch.Size([1, 5, 4, 768]))

In [5]:
12 * 768

9216

In [6]:
4 * 768

3072

In [7]:
k_repeated = torch.repeat_interleave(k, 3, dim=-2)
k_repeated.shape

torch.Size([1, 5, 12, 768])

In [8]:
tmp = torch.randint(0, 11, (1, 3, 5))
tmp

tensor([[[ 0,  3,  1, 10,  3],
         [ 0,  5,  8,  3,  3],
         [ 2,  0,  9,  7, 10]]])

In [9]:
tmp = torch.repeat_interleave(tmp, 3, dim=-2)

In [10]:
tmp[0].shape
tmp

tensor([[[ 0,  3,  1, 10,  3],
         [ 0,  3,  1, 10,  3],
         [ 0,  3,  1, 10,  3],
         [ 0,  5,  8,  3,  3],
         [ 0,  5,  8,  3,  3],
         [ 0,  5,  8,  3,  3],
         [ 2,  0,  9,  7, 10],
         [ 2,  0,  9,  7, 10],
         [ 2,  0,  9,  7, 10]]])

In [11]:
torch.ones(9, 5)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [12]:
mask = torch.triu(torch.ones(9, 5), diagonal=1).bool()
mask

tensor([[False,  True,  True,  True,  True],
        [False, False,  True,  True,  True],
        [False, False, False,  True,  True],
        [False, False, False, False,  True],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False]])

In [13]:
tmp2 = torch.masked_fill(tmp, mask, 1000)
tmp2

tensor([[[   0, 1000, 1000, 1000, 1000],
         [   0,    3, 1000, 1000, 1000],
         [   0,    3,    1, 1000, 1000],
         [   0,    5,    8,    3, 1000],
         [   0,    5,    8,    3,    3],
         [   0,    5,    8,    3,    3],
         [   2,    0,    9,    7,   10],
         [   2,    0,    9,    7,   10],
         [   2,    0,    9,    7,   10]]])

In [22]:
seq_len = 9
chunk_size = 3

mask = torch.zeros(seq_len, seq_len)
mask

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [23]:
for i in range(0, seq_len, chunk_size):
    print(f"Working at index: {i}")
    end = min(i + chunk_size, seq_len)
    mask[i:end, i:end] = 1
    print(f"Range: {i}-{end}")

mask

Working at index: 0
Range: 0-3
Working at index: 3
Range: 3-6
Working at index: 6
Range: 6-9


tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1.]])

In [ ]:
# Also apply causal within chunks
mask = mask * torch.tril(torch.ones(seq_len, seq_len))

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1.]])

In [25]:
torch.tril(torch.ones(5, 5))

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])